# Testing the applicative laws?

Now we have a problem, where we have several laws we want to adhere to, but testing that a property holds for *every* possible function is tricky. Fortunately, there is another definition for applicative, called monoidal.

Definition for monoidal looks like this:

For functor F, lets have 
* a empty F[Unit]
* function combine(F[a], F[b]) = F[(a,b)], that combines functor of a and functor of b into functor of tuples of a and b

Borrowing a trick from discrete math, we can have define Unit as an empty tuple.

In [61]:
Unit = ()

We can define both empty and combine with pure and apply:

In [63]:
class Applicative:
    def pure(self, val):
        raise NotImplementedError();
    
    def apply(self, fn, val):
        raise NotImplementedError();
    
    def empty(self):
        return self.pure(Unit)
    
    def combine(self, fa, fb):
        return self.apply(self.apply(self.pure(lambda x: lambda y: (x, y)),fa),fb)
    
    def map(self, f, x):
        return apply(pure(f),x)
    
    def lift(self, fn):
        def lifted(arg0, *args):
            result = self.apply(self.pure(curry(len(args)+1, fn)),arg0)
            for a in args:
                result = self.apply(result, a)
            return result
        return lifted

Now we can take the alternative laws for monoidal instances taken from [Applicative programming with effects](http://openaccess.city.ac.uk/13222/1/Applicative-final.pdf):

* naturality of `combine`: map(product(f,g), combine(u,v)) = combine(map(f,u), map(g,v))
* left identity: map(second, combine(Unit, v)) = v
* right identity: map(first, combine(v, Unit)) = v
* map(assoc, combine(u, combine(v, w)) = combine(combine(u, v), w))

We still need to define product, first, second and assoc:

In [71]:
def product(f,g):
    def p(a_tuple):
        a,b = a_tuple
        return (f(a),g(b))
    return p

def first(a_tuple):
    a,b = a_tuple
    return a

def second(a_tuple):
    a,b = a_tuple
    return b

def assoc(nested):
    a,(b,c) = nested
    return ((a,b),c)

Now we can 